In [1]:
from datetime import datetime
from scipy import stats

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import os
import pytz

import MetaTrader5 as mt5

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [2]:
# mt5 directory
mt5dir = "C:/Program Files/MetaTrader 5 IC Markets (SC) - 1"
# mt5dir = "C:/Program Files/Pepperstone MetaTrader 5"
mt5loc = f"{mt5dir}/terminal64.exe"

# connect to MetaTrader 5
if not mt5.initialize(mt5loc):
    print("initialize() failed")
    mt5.shutdown()
    
# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=True, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=3091, maxbars=100000000, codepage=0, ping_last=254472, community_balance=0.0, retransmission=0.0, company='Raw Trading Ltd', name='MetaTrader 5 IC Markets (SC)', language='English', path='C:\\Program Files\\MetaTrader 5 IC Markets (SC) - 1', data_path='C:\\Users\\nikki\\AppData\\Roaming\\MetaQuotes\\Terminal\\90EE3D256080A6E3EBC396D26D7AE724', commondata_path='C:\\Users\\nikki\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 3091, '22 Oct 2021')


In [3]:
# Get the symbols from IC Markets MT5, excluding equity stocks
symbols = mt5.symbols_get("*, !*.*")

product_columns = [
    'name',
    'description',
    'path',
    'currency_base',
    'currency_profit',
    'currency_margin',
    'bid',
    'ask',
    'spread',
#     'last',
    'trade_contract_size',
    'volume_min',
    'volume_max',
    'volume_step',
    'trade_tick_value',
    'trade_tick_size',
    'margin_initial',
#     'margin_maintenance',
    'swap_long',
    'swap_short'
]


product_df = pd.DataFrame(columns=product_columns)

for symbol in symbols:
    product_df = product_df.append(
        pd.Series(
            [
                symbol.name,
                symbol.description,
                symbol.path,
                symbol.currency_base,
                symbol.currency_profit,
                symbol.currency_margin,
                symbol.bid,
                symbol.ask,
                symbol.ask - symbol.bid,
#                 symbol.last,
                symbol.trade_contract_size,
                symbol.volume_min,
                symbol.volume_max,
                symbol.volume_step,
                symbol.trade_tick_value,
                symbol.trade_tick_size,
                symbol.margin_initial,
#                 symbol.margin_maintenance,
                symbol.swap_long,
                symbol.swap_short
            ],
            index=product_columns
        ),
        ignore_index=True
    )

product_df

,name,description,path,currency_base,currency_profit,currency_margin,bid,ask,spread,trade_contract_size,volume_min,volume_max,volume_step,trade_tick_value,trade_tick_size,margin_initial,swap_long,swap_short
0,EURUSD,Euro vs US Dollar,Forex\Majors\EURUSD,EUR,USD,EUR,1.13000,1.13000,0.00000,100000.0,0.01,200.0,0.01,1.000000,0.00001,100000.0,-3.80,0.28
1,GBPUSD,Great Britain Pound vs US Dollar,Forex\Majors\GBPUSD,GBP,USD,GBP,1.35392,1.35392,0.00000,100000.0,0.01,200.0,0.01,1.000000,0.00001,100000.0,-2.85,-3.27
2,USDCHF,US Dollar vs Swiss Franc,Forex\Majors\USDCHF,USD,CHF,USD,0.92224,0.92224,0.00000,100000.0,0.01,200.0,0.01,1.084316,0.00001,100000.0,0.07,-3.76
3,USDJPY,US Dollar vs Japanese Yen,Forex\Majors\USDJPY,USD,JPY,USD,115.82900,115.82900,0.00000,100000.0,0.01,200.0,0.01,0.863342,0.00100,100000.0,-1.61,-3.26
4,USDCAD,US Dollar vs Canadian Dollar,Forex\Majors\USDCAD,USD,CAD,USD,1.27234,1.27237,0.00003,100000.0,0.01,200.0,0.01,0.785935,0.00001,100000.0,-2.53,-2.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,JGB10Y_H2,Japan JGB 10 YR - March 22 CFD,Bonds CFDs\JGB10Y_H2,USD,JPY,JPY,150.90000,150.94000,0.04000,100.0,1.00,100.0,1.00,0.008633,0.01000,0.0,0.00,0.00
129,EURSCA_H2,Euro Schatz - March 22 CFD,Bonds CFDs\EURSCA_H2,USD,EUR,EUR,112.01000,112.02000,0.01000,100.0,1.00,100.0,1.00,1.130000,0.01000,0.0,0.00,0.00
130,ITB10Y_H2,Euro BTP Italian 10 YR - March 22 CFD,Bonds CFDs\ITB10Y_H2,USD,EUR,EUR,145.73000,145.75000,0.02000,100.0,1.00,100.0,1.00,1.130000,0.01000,0.0,0.00,0.00
131,DE40,Germany 40 Index,Indices\Indices Spot\Major Spot Indices\DE40,USD,EUR,EUR,15981.20000,15981.70000,0.50000,1.0,0.10,250.0,0.10,0.011300,0.01000,0.0,-0.83,-1.32


In [4]:
product_df.to_clipboard()

In [6]:
product_df[product_df['volume_min'] != product_df['volume_step']]

,name,description,path,currency_base,currency_profit,currency_margin,bid,ask,spread,trade_contract_size,volume_min,volume_max,volume_step,trade_tick_value,trade_tick_size,margin_initial,swap_long,swap_short
87,DSHUSD,Dash (USD),Crypto\DSHUSD,USD,USD,USD,126.67000,128.77000,2.10000,1.0,1.0,100.0,0.01,0.0100,0.01000,0.0,-15.0,-15.0
88,XRPUSD,Ripple (USD),Crypto\XRPUSD,USD,USD,USD,0.75840,0.76890,0.01050,1.0,1.0,100.0,0.01,0.0001,0.00010,0.0,-15.0,-15.0
89,EOSUSD,EOS (USD),Crypto\EOSUSD,USD,USD,USD,2.86000,2.87300,0.01300,1.0,1.0,100.0,0.01,0.0001,0.00010,0.0,-15.0,-15.0
122,XTZUSD,Tezos (USD),Crypto\XTZUSD,USD,USD,USD,4.29284,4.30810,0.01526,10.0,1.0,100.0,0.01,0.0001,0.00001,0.0,-15.0,-15.0
123,DOGUSD,Dogecoin USD,Crypto\DOGUSD,USD,USD,USD,0.15601,0.15688,0.00087,100.0,1.0,100.0,0.01,0.0010,0.00001,0.0,-15.0,-15.0
125,UNIUSD,Uniswap (USD),Crypto\UNIUSD,USD,USD,USD,16.06540,16.12100,0.05560,1.0,1.0,100.0,0.01,0.0001,0.00010,0.0,-15.0,-15.0
126,ADAUSD,Cardano (USD),Crypto\ADAUSD,USD,USD,USD,1.23999,1.24243,0.00244,10.0,1.0,100.0,0.01,0.0001,0.00001,0.0,-15.0,-15.0


In [7]:
product_df['name']

0         EURUSD
1         GBPUSD
2         USDCHF
3         USDJPY
4         USDCAD
         ...    
128    JGB10Y_H2
129    EURSCA_H2
130    ITB10Y_H2
131         DE40
132      MidDE50
Name: name, Length: 133, dtype: object

In [8]:
product_df['currency_profit'].unique()

array(['USD', 'CHF', 'JPY', 'CAD', 'NZD', 'GBP', 'AUD', 'SGD', 'DKK',
       'HKD', 'NOK', 'PLN', 'SEK', 'TRY', 'ZAR', 'CNH', 'CZK', 'HUF',
       'MXN', 'RUB', 'THB', 'EUR'], dtype=object)

In [9]:
product_df.to_clipboard()

# Download Data

In [11]:
# Create list of symbols to download
symbols = product_df['name']

# How many ohlc data to download
lookback = 100
# lookback = 10

# Set which timeframes to download
tf_dict = {
    'd1': mt5.TIMEFRAME_D1,
    'h4': mt5.TIMEFRAME_H4,
    'h1': mt5.TIMEFRAME_H1,
    'm30': mt5.TIMEFRAME_M30,
    'm15': mt5.TIMEFRAME_M15,
    'm5': mt5.TIMEFRAME_M5,
    'm1': mt5.TIMEFRAME_M1
}

cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

In [12]:
ohlcv = {}
tf_mt5 = mt5.TIMEFRAME_D1
for ticker in symbols:
    ohlcv[ticker] = pd.DataFrame(
        mt5.copy_rates_from_pos(
            ticker,
            tf_mt5,
            0,
            lookback
        )
    )
    try:
        ohlcv[ticker]['time'] = pd.to_datetime(
            ohlcv[ticker]['time'],
            unit='s'
        )
        # ohlcv[ticker] = ohlcv[ticker].tz_localize('utc')
        ohlcv[ticker] = ohlcv[ticker].set_index('time')
    except KeyError:
        print(f'KeyError on {ticker}')
    except ValueError:
        print(f'ValueError on {ticker}')

In [13]:
len(ohlcv.keys())

133

# Calculate Volatility

In [14]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

    # alternative caltulation using ewma
#     return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [15]:
# Create an empty DataFrame to store score

ins_risk_columns = [
    'ticker',
    'last_date',
    'ins_risk'
]

ins_risk_table = pd.DataFrame(columns=ins_risk_columns)

# How many (series) candles back for std dev calculation?
vola_window = 24

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, ohlc in ohlcv.items():
    try:
        ins_risk = volatility(ohlc['close'], vola_window) * 16
        last_date = ohlc.index[-1]
        ins_risk_table = ins_risk_table.append(
            pd.Series(
                [
                    ticker,
                    last_date,
                    ins_risk
                ],
                index=ins_risk_columns
            ),
            ignore_index=True
        )
    except KeyError:
        print(f'KeyError on {ticker}')

ins_risk_table

,ticker,last_date,ins_risk
0,EURUSD,2022-01-07,0.058893
1,GBPUSD,2022-01-07,0.054550
2,USDCHF,2022-01-07,0.053282
3,USDJPY,2022-01-07,0.036457
4,USDCAD,2022-01-07,0.078303
...,...,...,...
128,JGB10Y_H2,2022-01-07,NaN
129,EURSCA_H2,2022-01-07,NaN
130,ITB10Y_H2,2022-01-07,NaN
131,DE40,2022-01-07,0.148401


In [16]:
ins_risk_table.to_clipboard()

In [20]:
product_df['ins_risk'] = ins_risk_table['ins_risk']